In [39]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import stockdata as sd

In [60]:
def getindexsymbolsinsectors(indexname):
    query = f"""
    SELECT indexname, name, a.symbol, 1 x
    from nseindices a
        join symbols b on a.symbol = b.symbol
    where 1 = 1
    and (indextype = 'Sector' or indexname = '{indexname}')
    """
    df = sd.getdata(query)
    df = pd.pivot_table(df, index=['symbol', 'name'], columns='indexname')
    df.columns = df.columns.droplevel()
    df.reset_index(inplace=True)
    df.fillna(0, inplace=True)
    cols = df.columns[df.dtypes.eq('float64')]
    df[cols] = df[cols].apply(pd.to_numeric, errors='ignore', axis=1, downcast='integer')
    df.columns = map(lambda x: x.replace('Nifty ', '') if x != indexname else x, df.columns)
    return df

In [61]:
df1 = getindexsymbolsinsectors('Nifty 50')
df2 = getindexsymbolsinsectors('Nifty 100')
with pd.ExcelWriter('Nifty Index Symbols.xlsx',) as writer:
    df1.to_excel(writer, sheet_name='Nifty 50', index=False, freeze_panes=(1,3))
    df2.to_excel(writer, sheet_name='Nifty 100', index=False, freeze_panes=(1,3))

In [62]:
df = pd.read_csv('https://www1.nseindia.com/products/content/derivatives/equities/fo_underlying_home.htm')

TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond